In [13]:
import gymnasium as gym
import torch
import numpy as np
import dedalus.public as d3
import logging
import h5py
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from rbc_env import DedalusRBC_Env

In [14]:
#env = make_vec_env(DedalusRBC_Env, n_envs=30, seed=0, vec_env_cls=SubprocVecEnv)
gym.envs.register(id='rbc', entry_point=DedalusRBC_Env)
env = gym.make('rbc', render_mode="human")

/home/alexskillen/RB_DRL/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment rbc already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [15]:
model = PPO.load("logs/rl_model_450000_steps", env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [5]:
from stable_baselines3.common.evaluation import evaluate_policy
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=3)

In [16]:
vec_env = model.get_env()
obs = vec_env.reset()
for i in range(256):
    action, _ = model.predict(obs, deterministic=True)
    obs, _, _, _ = vec_env.step(action)
    T = env.unwrapped.problem.variables[1]['g']
    fig, ax = plt.subplots()
    c = ax.imshow(np.transpose(T), aspect=1/np.pi, origin="lower", vmin=0., vmax=1.4)
    fig.colorbar(c)
    plt.title('$Nu=$'+str(np.round(env.unwrapped.fp.properties['Nu']['g'].flatten()[-1], 2)))
    plt.savefig('figs/'+str(i)+'.png')
    plt.close()

In [17]:
env.unwrapped.solver.stop_sim_time *= 2
for i in range(256):
    action, _ = model.predict(obs)
    obs, _, _, _ = vec_env.step(action*0)
    T = env.unwrapped.problem.variables[1]['g']
    fig, ax = plt.subplots()
    c = ax.imshow(np.transpose(T), aspect=1/np.pi, origin="lower", vmin=0., vmax=1.4)
    fig.colorbar(c)
    plt.title('$Nu=$'+str(np.round(env.unwrapped.fp.properties['Nu']['g'].flatten()[-1], 2)))
    plt.savefig('figs/'+str(248+i)+'.png')
    plt.close()

In [11]:
model.predict(obs)

(array([[-1.        ,  1.        ,  0.06012079, -1.        , -1.        ,
         -1.        , -0.9429523 , -0.11217751, -1.        , -0.22566158]],
       dtype=float32),
 None)